In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [2]:
#Khởi tạo mediapie
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Hàm tính khoảng cách

In [3]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle


test

In [4]:
import numpy as np

# Tọa độ có z khác 0
pointA = (1, 0, 1)
pointB = (0, 0, 0)
pointC = (0, 1, 2)

angle = calculate_angle(pointA, pointB, pointC)
print(f"Góc giữa 3 điểm là: {angle:.2f} độ") # 50.77 độ 


Góc giữa 3 điểm là: 71.57 độ


## 4. Xuống quá xâu 
Góc giữa hông, vai, đầu gối nhỏ 


In [5]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [6]:
#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    # # # Lấy Z của NOSE và HIP làm chuẩn
    # nose_z = landmarks[mp_pose.PoseLandmark.NOSE.value].z
    # hip_z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
    # scale_factor = abs(nose_z - hip_z) if abs(nose_z - hip_z) > 0 else 1  # Tránh chia cho 0

    # # # Hàm chuẩn hóa Z theo khoảng cách mũi, hông
    # def rescale_z(z):
    #     return (z - nose_z) / (scale_factor*2)

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        # data.append([keypoint.x, keypoint.y, rescale_z(keypoint.z), keypoint.visibility])
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


Xem data chuẩn

In [27]:
#Làm sạch dữ liệu dataframe trước khi thêm dữ liệu mới
df = pd.DataFrame(columns=["video", "stage", "shoulder_hip_ankle", "hip_angle"])

In [26]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [ ]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def process_frame(Video_folder, Video_name):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            shoulder_hip_ankle_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                predicted_class = trained_model.predict(X)[0]
                prediction_probability = trained_model.predict_proba(X)[0]

                current_stage ="middle"
                if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.8 thì mới cập nhật trạng thái
                    if predicted_class == 0:
                        current_stage = "down"
                    elif current_stage == "middle" and predicted_class == 1:
                        current_stage = "up"
                else:
                    current_stage = "middle"

                #Góc giữa vai-hông-chân phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_shoulder_hip_ankle_angle = calculate_angle(right_shoulder, right_hip, right_ankle)

                #Góc giữa vai-hông-chân trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]

                left_shoulder_hip_ankle_angle = calculate_angle(left_shoulder, left_hip, left_ankle)

                #Trung bình 2 góc vai-hông-chân
                shoulder_hip_ankle_angle = (right_shoulder_hip_ankle_angle + left_shoulder_hip_ankle_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "shoulder_hip_ankle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_hip_ankle_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            df.loc[len(df)] = [Video_name, current_stage, shoulder_hip_ankle_angle, hip_angle]

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            plt.imshow(image)
            plt.axis("off")  # Ẩn trục tọa độ
            plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [36]:
import os

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_folder = "Data/Train/Xuong_qua_xau"
video_folder = "Data/Train/Gap_lung"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

df = pd.DataFrame(columns=["video", "stage", "shoulder_hip_ankle_angle", "hip_angle"])

for video in video_files:
    process_frame(video_folder, video)

In [37]:
df 

,video,stage,shoulder_hip_ankle_angle,hip_angle
0,20250228_095350000_iOS.mp4,middle,168.462426,173.092937
1,20250228_095350000_iOS.mp4,middle,167.512652,172.753557
2,20250228_095350000_iOS.mp4,middle,167.111901,172.499327
3,20250228_095350000_iOS.mp4,middle,167.261776,172.469905
4,20250228_095350000_iOS.mp4,middle,167.488403,172.485154
...,...,...,...,...
1972,VID_20250228_165753.mp4,middle,172.274025,173.419985
1973,VID_20250228_165753.mp4,middle,169.865465,174.302062
1974,VID_20250228_165753.mp4,middle,168.845898,174.314011
1975,VID_20250228_165753.mp4,middle,169.261350,174.263004


In [38]:
#Tính toán tỉ lệ giữa vai và chân
df["ratio"] = df["hip_angle"] / df["shoulder_hip_ankle_angle"]
df

,video,stage,shoulder_hip_ankle_angle,hip_angle,ratio
0,20250228_095350000_iOS.mp4,middle,168.462426,173.092937,1.027487
1,20250228_095350000_iOS.mp4,middle,167.512652,172.753557,1.031287
2,20250228_095350000_iOS.mp4,middle,167.111901,172.499327,1.032238
3,20250228_095350000_iOS.mp4,middle,167.261776,172.469905,1.031138
4,20250228_095350000_iOS.mp4,middle,167.488403,172.485154,1.029833
...,...,...,...,...,...
1972,VID_20250228_165753.mp4,middle,172.274025,173.419985,1.006652
1973,VID_20250228_165753.mp4,middle,169.865465,174.302062,1.026118
1974,VID_20250228_165753.mp4,middle,168.845898,174.314011,1.032385
1975,VID_20250228_165753.mp4,middle,169.261350,174.263004,1.029550


In [39]:
#Phân tích bộ data
df.describe()


,shoulder_hip_ankle_angle,hip_angle,ratio
count,1977.000000,1977.000000,1977.000000
mean,150.223554,118.602209,0.769146
std,18.145365,40.749396,0.182824
min,112.617828,57.456165,0.478989
25%,133.151427,79.850948,0.601556
50%,150.398338,110.106594,0.738343
75%,168.554031,168.555970,0.977573
max,174.882478,174.314011,1.056138


In [40]:
#Phân tích bộ data theo stage
df_group = df.groupby("stage").describe()
# Lọc chỉ các cột có mean, min, max
df_filtered = df_group.loc[:, (slice(None), ['count', 'mean', 'min', 'max'])]

# Hiển thị kết quả
print(df_filtered)

       shoulder_hip_ankle_angle                                     hip_angle  \
                          count        mean         min         max     count   
stage                                                                           
down                      506.0  129.003850  112.617828  151.620917     506.0   
middle                   1379.0  156.762059  112.798845  174.882478    1379.0   
up                         92.0  168.925436  161.271495  174.860003      92.0   

                                             ratio                      \
              mean         min         max   count      mean       min   
stage                                                                    
down     72.930463   57.456165  114.799681   506.0  0.563908  0.478989   
middle  131.857380   64.326750  174.314011  1379.0  0.828163  0.557184   
up      171.113335  168.252032  173.562068    92.0  1.013344  0.962881   

                  
             max  
stage             
down    0.7

Down: ratio min: 0.52
Middle: ratio min: 0.6
Up: ratio min: 0.92

In [41]:
#test
df_false = df[((df["ratio"] < 0.52) & (df["stage"] == "down")) | 
              ((df["ratio"] < 0.6) & (df["stage"] == "middle"))] #Tỷ lệ góc hông so với góc cơ thể nhỏ nhất theo data đúng
df_false


,video,stage,shoulder_hip_ankle_angle,hip_angle,ratio
270,20250228_095518000_iOS.mp4,down,123.880022,64.219114,0.518398
271,20250228_095518000_iOS.mp4,down,123.362488,63.212593,0.512413
272,20250228_095518000_iOS.mp4,down,123.255415,62.664830,0.508414
273,20250228_095518000_iOS.mp4,down,122.895677,62.155471,0.505758
274,20250228_095518000_iOS.mp4,down,122.756723,62.214757,0.506813
...,...,...,...,...,...
1899,VID_20250228_165753.mp4,down,129.246125,61.907506,0.478989
1900,VID_20250228_165753.mp4,down,133.128719,65.202542,0.489771
1901,VID_20250228_165753.mp4,down,134.509361,66.402223,0.493662
1902,VID_20250228_165753.mp4,down,134.932839,66.772030,0.494854


In [42]:
df_false.describe()

,shoulder_hip_ankle_angle,hip_angle,ratio
count,178.000000,178.000000,178.000000
mean,128.703063,69.542897,0.539555
std,6.251893,6.951017,0.037400
min,112.798845,57.456165,0.478989
25%,124.990488,63.333768,0.505735
50%,128.241191,70.707598,0.557957
75%,135.221092,76.366676,0.572675
max,138.575151,81.761349,0.599982


# Kết luận:
